In [1]:
import matplotlib.pyplot as plt 
import scipy.misc
import json

import numpy as np
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, Cropping2D
from keras.layers import MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation, merge, GlobalAveragePooling2D
from keras.layers.merge import Add, Multiply, Average, Maximum, Concatenate, Dot
from keras.models import Model
from keras.utils.layer_utils import print_summary
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from scipy.misc import imread

from keras import __version__ as kv
from scipy import __version__ as sv
from matplotlib import __version__ as mv
from numpy import __version__ as nv

print(kv)
print(sv) 
print(mv) 
print(nv) 

Using TensorFlow backend.


2.0.7
0.19.0
2.0.0
1.12.1


In [2]:
# Local files
import utils_python3
#reload(utils_python3)

# Data

### Config

In [3]:
trained_model_dir = '/home/ubuntu/data/sar/training_crops_20170829/trained_models/2.0-as-Python3_FCN_all_data/'
train_dir = '/home/ubuntu/data/sar/training_crops_20170829/train/50x50/'
valid_dir = '/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/'

In [4]:
input_shape = (50, 50, 1)
num_classes = 3

### Split training_crops json into train and validate

Done in 1.0-mph_kjd_as-Python3_binary_classifier_all_data_turbine.ipynb

In [5]:
# utils_python3.split_all_json_into_train_validate('oil_and_gas_infrastructure')
# utils_python3.split_all_json_into_train_validate('other')
# utils_python3.split_all_json_into_train_validate('turbine')

1756
501
2234
501
6198
501


### Covert tif2png

Done in 1.0-mph_kjd_as-Python3_binary_classifier_all_data_turbine.ipynb

In [5]:
# utils_python3.tif2png(src_dir=train_dir+"oil_and_gas_infrastructure/", dest_dir=train_dir+"oil_and_gas_infrastructure/")
# utils_python3.tif2png(src_dir=train_dir+"other/", dest_dir=train_dir+"other/")
# utils_python3.tif2png(src_dir=train_dir+"turbine/", dest_dir=train_dir+"turbine/")

/home/ubuntu/data/sar/training_crops_20170829/train/50x50/oil_and_gas_infrastructure/ 1756
/home/ubuntu/data/sar/training_crops_20170829/train/50x50/oil_and_gas_infrastructure/ 1756 3512
/home/ubuntu/data/sar/training_crops_20170829/train/50x50/other/ 2234
/home/ubuntu/data/sar/training_crops_20170829/train/50x50/other/ 2234 4468
/home/ubuntu/data/sar/training_crops_20170829/train/50x50/turbine/ 6198
/home/ubuntu/data/sar/training_crops_20170829/train/50x50/turbine/ 6198 12396


In [6]:
# utils_python3.tif2png(src_dir=valid_dir+"oil_and_gas_infrastructure/", dest_dir=valid_dir+"oil_and_gas_infrastructure/")
# utils_python3.tif2png(src_dir=valid_dir+"other/", dest_dir=valid_dir+"other/")
# utils_python3.tif2png(src_dir=valid_dir+"turbine/", dest_dir=valid_dir+"turbine/")

/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/oil_and_gas_infrastructure/ 501
/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/oil_and_gas_infrastructure/ 501 1002
/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/other/ 501
/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/other/ 501 1002
/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/turbine/ 501
/home/ubuntu/data/sar/training_crops_20170829/validate/50x50/turbine/ 501 1002


### Add dist2land feature

In [5]:
train_crops, train_filename, train_feature, train_class, \
valid_crops, valid_filename, valid_feature, valid_class = utils_python3.add_dist2land_training_crops_20170829_multiclass("50x50")

In [6]:
print("training crops:", len(train_crops))
print("training features:", len(train_feature))
print("validation crops:", len(valid_crops))
print("validation features:", len(valid_feature))
print(type(train_crops), type(train_crops[0]), train_crops[0].shape)

training crops: 10188
training features: 10188
validation crops: 1503
validation features: 1503
<class 'list'> <class 'numpy.ndarray'> (50, 50)


In [7]:
# Reshape for keras format
np_train_class = np.array(train_class)
np_train_filename = np.array(train_filename)
np_train_crops = np.array(train_crops)
np_train_feature = np.array(train_feature)

np_valid_class = np.array(valid_class)
np_valid_filename = np.array(valid_filename)
np_valid_crops = np.array(valid_crops)
np_valid_feature = np.array(valid_feature)

np_train_crops = np.expand_dims(np_train_crops, axis=3)
np_train_feature = np.expand_dims(np_train_feature, axis=1)

np_valid_crops = np.expand_dims(np_valid_crops, axis=3)
np_valid_feature = np.expand_dims(np_valid_feature, axis=1)

In [8]:
print(np_train_crops.shape)
print(np_train_feature.shape)
print(np_train_class.shape)
print(np_valid_crops.shape)
print(np_valid_feature.shape)
print(np_valid_class.shape)

(10188, 50, 50, 1)
(10188, 1)
(10188, 3)
(1503, 50, 50, 1)
(1503, 1)
(1503, 3)


# Experiments

with distance

In [9]:
p = 0

classifier_input = Input(shape=input_shape)
dist2land_input = Input(shape=(1,))

x = Conv2D(32, (3, 3), activation='relu', padding='same')(classifier_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

# # Following based on https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/lesson7.ipynb
x = BatchNormalization(axis=-1)(x)
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Conv2D(64,(3,3), activation='relu', padding='same')(x)
x = BatchNormalization(axis=-1)(x)
x = MaxPooling2D()(x)   # REMOVED MAX POOLING FOR VISUALISATION
x = Conv2D(3,(3,3), padding='same')(x)
x = Dropout(p)(x)
x = GlobalAveragePooling2D()(x)
m = Add()([dist2land_input, x])
out = Activation('softmax')(m)


model_with_distance = Model(inputs=[classifier_input, dist2land_input], outputs=out)
model_with_distance.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model_with_distance.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 50, 50, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 50, 50, 32)    320         input_1[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 25, 25, 32)    0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 25, 25, 64)    18496       max_pooling2d_1[0][0]            
___________________________________________________________________________________________

In [11]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model_with_distance).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [12]:
lr = 0.001
K.set_value(model_with_distance.optimizer.lr, lr)

model_with_distance.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 10188 samples, validate on 1503 samples
Epoch 1/10
10188/10188 [==============================] - 7s - loss: 0.2610 - acc: 0.9086 - val_loss: 0.7323 - val_acc: 0.7864
Epoch 2/10
10188/10188 [==============================] - 6s - loss: 0.1483 - acc: 0.9484 - val_loss: 0.1825 - val_acc: 0.9261
Epoch 3/10
10188/10188 [==============================] - 6s - loss: 0.1117 - acc: 0.9618 - val_loss: 0.1827 - val_acc: 0.9222
Epoch 4/10
10188/10188 [==============================] - 6s - loss: 0.0885 - acc: 0.9687 - val_loss: 0.1706 - val_acc: 0.9508
Epoch 5/10
10188/10188 [==============================] - 6s - loss: 0.0766 - acc: 0.9733 - val_loss: 0.1800 - val_acc: 0.9408
Epoch 6/10
10188/10188 [==============================] - 6s - loss: 0.0667 - acc: 0.9766 - val_loss: 0.1151 - val_acc: 0.9607
Epoch 7/10
10188/10188 [==============================] - 6s - loss: 0.0552 - acc: 0.9807 - val_loss: 0.1103 - val_acc: 0.9601
Epoch 8/10
10188/10188 [==============================] - 6s -

In [14]:
lr = 0.0001
K.set_value(model_with_distance.optimizer.lr, lr)

model_with_distance.fit([np_train_crops, np_train_feature], np_train_class,
          batch_size=32,
          epochs=10,
          validation_data=([np_valid_crops, np_valid_feature], np_valid_class))

Train on 10188 samples, validate on 1503 samples
Epoch 1/10
10188/10188 [==============================] - 6s - loss: 3.3022e-04 - acc: 1.0000 - val_loss: 0.0839 - val_acc: 0.9827
Epoch 2/10
10188/10188 [==============================] - 6s - loss: 2.7385e-04 - acc: 1.0000 - val_loss: 0.0897 - val_acc: 0.9800
Epoch 3/10
10188/10188 [==============================] - 6s - loss: 2.1567e-04 - acc: 1.0000 - val_loss: 0.0840 - val_acc: 0.9820
Epoch 4/10
10188/10188 [==============================] - 6s - loss: 1.7408e-04 - acc: 0.9999 - val_loss: 0.0844 - val_acc: 0.9847
Epoch 5/10
10188/10188 [==============================] - 6s - loss: 1.0817e-04 - acc: 1.0000 - val_loss: 0.0858 - val_acc: 0.9840
Epoch 6/10
10188/10188 [==============================] - 6s - loss: 1.1829e-04 - acc: 1.0000 - val_loss: 0.0952 - val_acc: 0.9820
Epoch 7/10
10188/10188 [==============================] - 6s - loss: 3.6400e-05 - acc: 1.0000 - val_loss: 0.0909 - val_acc: 0.9827
Epoch 8/10
10188/10188 [==========